In [42]:
import requests
import zipfile
import xml.etree.ElementTree as ET
import pandas as pd

## OASIS API Call Test

Docmentation is [here](http://www.caiso.com/Documents/OASIS-InterfaceSpecification_v5_1_1Clean_Fall2017Release.pdf). 

Note that the examples in the document use 2013 dates, but this may be too far in the past to work. Substituting it with 2021 worked for me. 

In [5]:
#test url 
url = "http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=20210919T07:00-0000&enddatetime=20210920T07:00-0000&version=1&market_run_id=DAM&node=DLAP_PGAE-APND"

In [46]:
test_request = requests.get(url)

In [48]:
test_request.headers['Content-Disposition']

'inline; filename=20210919_20210920_PRC_LMP_DAM_20211010_15_05_20_v1.zip;'

In [50]:
#extract zip file
with open('trythis.zip', 'wb') as code:
    code.write(test_request.content)

In [53]:
with zipfile.ZipFile('trythis.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

## Explore XML Data to convert to Pandas Dataframe

In [6]:
xml_data = ET.parse('data/20210919_20210920_PRC_LMP_DAM_20211010_15_05_20_v1.xml')

In [7]:
root = xml_data.getroot()

In [8]:
print(root.tag)

{http://www.caiso.com/soa/OASISReport_v1.xsd}OASISReport


In [9]:
root.attrib

{}

In [10]:
for child in root:
    print(child.tag, child.attrib)

{http://www.caiso.com/soa/OASISReport_v1.xsd}MessageHeader {}
{http://www.caiso.com/soa/OASISReport_v1.xsd}MessagePayload {}


In [11]:
[elem.tag for elem in root.iter()]

['{http://www.caiso.com/soa/OASISReport_v1.xsd}OASISReport',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}MessageHeader',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}TimeDate',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}Source',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}Version',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}MessagePayload',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}RTO',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}name',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT_ITEM',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT_HEADER',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}SYSTEM',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}TZ',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}REPORT',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}MKT_TYPE',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}UOM',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}INTERVAL',
 '{http://www.caiso.com/soa/OASISReport_v1.xsd}SEC_PER_INTERVAL',
 '{http://w

In [38]:
values = [repdata.text for repdata in root.iter('{http://www.caiso.com/soa/OASISReport_v1.xsd}VALUE')]
interval_start = [repdata.text for repdata in root.iter('{http://www.caiso.com/soa/OASISReport_v1.xsd}INTERVAL_START_GMT')]
interval_end = [repdata.text for repdata in root.iter('{http://www.caiso.com/soa/OASISReport_v1.xsd}INTERVAL_END_GMT')]
interval_num = [repdata.text for repdata in root.iter('{http://www.caiso.com/soa/OASISReport_v1.xsd}INTERVAL_NUM')]
opr_date = [repdata.text for repdata in root.iter('{http://www.caiso.com/soa/OASISReport_v1.xsd}OPR_DATE')]
resource_name = [repdata.text for repdata in root.iter('{http://www.caiso.com/soa/OASISReport_v1.xsd}RESOURCE_NAME')]

In [43]:
test_df = pd.DataFrame({'opr_date': opr_date,
                        'interval_start': interval_start,
                        'interval_end': interval_end, 
                        'interval_num': interval_num,
                        'resource_name': resource_name,
                        'value': values})

In [44]:
test_df.head()

,opr_date,interval_start,interval_end,interval_num,resource_name,value
0,2021-09-19,2021-09-19T15:00:00-00:00,2021-09-19T16:00:00-00:00,9,DLAP_PGAE-APND,30.74893
1,2021-09-19,2021-09-20T03:00:00-00:00,2021-09-20T04:00:00-00:00,21,DLAP_PGAE-APND,74.19184
2,2021-09-19,2021-09-19T07:00:00-00:00,2021-09-19T08:00:00-00:00,1,DLAP_PGAE-APND,58.41894
3,2021-09-19,2021-09-19T08:00:00-00:00,2021-09-19T09:00:00-00:00,2,DLAP_PGAE-APND,56.26837
4,2021-09-19,2021-09-19T22:00:00-00:00,2021-09-19T23:00:00-00:00,16,DLAP_PGAE-APND,43.2608


'CAISO'